In [1]:
import pandas as pd
import numpy as np

In [ ]:
# Barcelona (RST) -> Paris (POI)
# NewYork (RST) -> Paris (POI)

In [2]:
def get_data(city_r, city_p):
    # Obtener el conjunto de datos para entrenar (comieron en r y fueron a p)
    top_n_pois = 50

    city_r = city_r.lower().replace(" ", "")
    city_p = city_p.lower().replace(" ", "")

    reviews_r_path = f"out/{city_r}/restaurants/reviews.pkl"
    reviews_p_path = f"out/{city_p}/pois/reviews.pkl"

    reviews_r = pd.read_pickle(reviews_r_path)
    reviews_p = pd.read_pickle(reviews_p_path)

    reviews_r = reviews_r[reviews_r["userId"]!=-1]
    reviews_p = reviews_p[reviews_p["userId"]!=-1]

    # Quedarse con los POIs más populares
    reviews_p_popular = reviews_p.groupby("itemId")["itemId"].count().sort_values(ascending=False).head(top_n_pois).index.values
    reviews_p = reviews_p[reviews_p["itemId"].isin(reviews_p_popular)]

    r_data = set(reviews_r["userId"].unique())
    p_data = set(reviews_p["userId"].unique())

    common_users = r_data.intersection(p_data)

    # ToDo: ¿Por que hay menos usuarios en el conjunto users.pkl que en reviews.pkl?
    # ToDo: En R, tienen que ser solo los comunes???
    reviews_r = reviews_r.loc[reviews_r["userId"].isin(common_users)]
    reviews_p = reviews_p.loc[reviews_p["userId"].isin(common_users)]

    out_data = []

    for rst_id, rst_data in reviews_r.groupby("itemId"):
        rst_users = rst_data["userId"].unique()
        poi_revws = reviews_p.loc[reviews_p["userId"].isin(rst_users)]
        poi_revws = poi_revws.groupby("userId")["itemId"].unique().reset_index()
        # ToDo: Ojo, que los usuarios de r van a más de un POI en p
        poiId, times = np.unique(np.concatenate(poi_revws["itemId"].values), return_counts=True)

        print(rst_id, len(rst_users), poiId )

    return reviews_r, reviews_p

dataset = get_data("barcelona", "paris")

693801 33 [  188149   188150   188467   188678   188679   188698   188709   189245
   189283   189284   189683   189687   190202   190685   191240   194171
   209760   265614   265635   292257   601755   654725   667302  1557378
  2397509 19457687]
693822 1 [188679 189245 190202 190685]
693855 11 [ 188150  188679  188698  188709  189193  189245  189284  189683  189687
  190202  190685  191240  209760  265635  314450  601755 1008070]
693967 54 [ 188149  188150  188467  188678  188679  188698  188709  189193  189245
  189283  189284  189683  189687  190202  190685  191240  194171  209760
  263032  265614  265635  292257  314440  314450  577423  601755  654725
 1557378 2397509]
697396 578 [  188149   188150   188467   188678   188679   188698   188709   189193
   189245   189283   189284   189683   189687   190202   190685   191240
   194171   209760   263032   265614   265635   292257   314440   314450
   527838   601755   654725   667302  1008070  1557378  1600380  1784427
  2397509  26